<a href="https://colab.research.google.com/github/vkjadon/llm/blob/main/hf_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We can use `AutoModel` class from the `transformers` library to download and cache the specific model architecture and weights using `from_pretrained()` method. The AutoModel class is a wrappers designed to fetch the appropriate model architecture for a given checkpoint.

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("bert-base-cased")

In this case AutoModel will fetch a BERT model on the basis of the checkpoint provided in the `from_pretrained()` method. It downloads and chches the model architechure (12 layers, 768 hidden size, 12 attention heads) and the weights from HuggingFace Hub.

However, we can use a specific model class directly in case we know the type of model we want to use for the checkpoint.

In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

## Loading and saving

We can use `save_pretrained()` method, to save the model's weights and architecture configuration.

In [ ]:
model.save_pretrained("bert")

This will save two file in the path provided in the `save_pretrained()`. The path will have two files `config.json` and `model.safetensors`.

The config.json file have necessary attributes needed to build the model architecture and some metadata. The `model.safetensors` is the state dictionary containing weights.

To reuse a saved model, use the from_pretrained() method again.

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("bert")

## Model Flow

Transformer models handle text by turning the inputs into numbers. Here we will look at exactly what happens when your text is processed by the tokenizer.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

encoded_input = tokenizer("Hello, I'm a single sentence!")
print(encoded_input)

We can decode the input IDs to get back the original text

In [ ]:
tokenizer.decode(encoded_input["input_ids"])

You’ll notice that the tokenizer has added special tokens — [CLS] and [SEP] — required by the model. Not all models need special tokens; they’re utilized when a model was pretrained with them

In [ ]:
encoded_input = tokenizer(["How are you?", "I'm fine, thank you!"])
print(encoded_input)

Let us ask the tokenizer to return tensors directly from PyTorch

In [ ]:
encoded_input = tokenizer(
    ["How are you?", "I'm fine, thank you!"],
    return_tensors="pt"
)
print(encoded_input)

When you ask the tokenizer to return PyTorch tensors (return_tensors="pt"), it needs all sequences to be of the same length to form a batch. To fix this, you need to explicitly tell the tokenizer to pad the shorter sequences so they all match the length of the longest one. You can do this by adding padding=True to your tokenizer call.

In [ ]:
encoded_input = tokenizer(
    ["How are you?", "I'm fine, thank you!"], padding=True, return_tensors="pt"
)
print(encoded_input)

Note that the padding tokens have been encoded into input IDs with ID 0, and they have an attention mask value of 0 as well. This is because those padding tokens shouldn't be analyzed by the model: they're not part of the actual sentence.

The tensors might get too big to be processed by the model. For instance, BERT was only pretrained with sequences up to 512 tokens, so it cannot process longer sequences. If you have sequences longer than the model can handle, you’ll need to truncate them with the truncation parameter:

In [36]:
encoded_input = tokenizer(
    "This is a very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very long sentence.",
    padding=True,
    truncation=True,
    return_tensors="pt",
)
print(encoded_input)

{'input_ids': tensor([[ 101, 1188, 1110,  170, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304,
         1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304,
         1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304,
         1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304,
         1304, 1304, 1304, 1304, 1304, 1263, 5650,  119,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]])}


Then the tokenizer truncates only if your input exceeds the model's default maximum length. For model_max_length for BERT is 512 tokens, for GPT-2 is 1024 for LLaMA it is 2048, 4096, etc.
We can explicitly set maximum length by using `max_length` parameter.

In [31]:
encoded_input = tokenizer(
    ["How are you?", "I'm fine, thank you!"],
    padding=True,
    truncation=True,
    max_length=5,
    return_tensors="pt",
)
print(encoded_input)

{'input_ids': tensor([[ 101, 1731, 1132, 1128,  102],
        [ 101,  146,  112,  182,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])}


In [ ]:
encoded_input = tokenizer("How are you?")
print(encoded_input["input_ids"])
tokenizer.decode(encoded_input["input_ids"])

In [ ]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]